<a href="https://colab.research.google.com/github/gseetha04/Transfer-learning/blob/main/Transferlearning_experiments_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##############Fine-tune on SCITE and test on SCITE####################################

In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, BertTokenizerFast
# sequence tagging model + training-related
from transformers import BertForTokenClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_name = "bert-base-cased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
data_train = pd.read_excel('/content/train_data.xlsx',sheet_name = "Sheet5")
data_train.head(4)

,sentences,Bio_label
0,"[""A"",""rare"",""and"",""incurable"",""congenital"",""di...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""E"",""O"",""O"",""..."
1,"[""The"",""soft"",""brown"",""and"",""beige"",""tones"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""O"",""O"",""E"",""..."
2,"[""The"",""JFK"",""baggage"",""system"",""malfunction"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""..."
3,"[""Cobalt"",""metal"",""fume"",""and"",""dust"",""cause"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""O""]"


In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
data_test

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."
4,"[""The"",""alleged"",""abuse"",""resulted"",""in"",""brui...","[""C"",""C"",""C"",""O"",""O"",""E"",""O"",""E"",""O"",""O"",""O""]"
...,...,...
186,"[""Thus"","","",""evaluating"",""capital"",""punishment...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."
187,"[""About"",""30"",""ducks"",""were"",""found"",""dead"",""i...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."
188,"[""He"",""created"",""and"",""advocated"",""-"",""flower""...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."
189,"[""Method"",""according"",""to"",""claim"",""1"","","",""ch...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_train[["sentences", "Bio_label"]].rename(columns={"sentences": "X", "Bio_label": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]


In [ ]:
X_raw_test

[['Various',
  'hormonal',
  ',',
  'bacterial',
  'and',
  'inflammatory',
  'disturbances',
  'cause',
  'acne'],
 ['A',
  'stereo',
  'buss',
  'outputs',
  'the',
  'stereo',
  'buss',
  'signal',
  ';',
  'a',
  'record',
  'buss',
  'outputs',
  'the',
  'record',
  'buss',
  'signal'],
 ['The',
  'incoming',
  'water',
  'caused',
  'a',
  'stain',
  'on',
  'the',
  'wall',
  'that',
  ',',
  'through',
  'the',
  'lens',
  'of',
  'pareidolia',
  ',',
  'looked',
  'like',
  'a',
  'bearded',
  'man',
  'with',
  'a',
  'big',
  'afro',
  'hairdo'],
 ['The',
  'genreal',
  'anesthetic',
  'cause',
  'unconsciousness',
  'and',
  'insensibility',
  'to',
  'paid',
  'and',
  'are',
  'used',
  'for',
  'major',
  'surgical',
  'procedures'],
 ['The',
  'alleged',
  'abuse',
  'resulted',
  'in',
  'bruises',
  'and',
  'swelling',
  'of',
  'the',
  'brain'],
 ["Obama's",
  'economic',
  'policies',
  'are',
  'turning',
  'into',
  'a',
  'global',
  'disaster'],
 ['These',
  

In [ ]:
y_raw_test

[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E',
  'O',
  'C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'E', 'O', 'E', 'O', 'O', 'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'E', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'O', 'O', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
id2tag

{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_dev_raw, X_dev_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_raw_test, X_raw_test]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())


    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(X_train_raw)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(X_raw_test)

y_train = encode_tags(tags=y_train_raw, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.976200,0.887334
2,0.659700,0.549919
3,0.229200,0.198399


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(13561,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(13561,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,0,-100
1,3,3
2,3,3
3,0,-100
4,0,-100
...,...,...
13556,0,-100
13557,0,-100
13558,0,-100
13559,0,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,3,3
2,3,3
5,0,3
6,3,3
7,0,3
...,...,...
13519,0,3
13520,3,3
13521,0,0
13522,0,0


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'CE', 3: 'C'}


In [ ]:
targetname = ['O','E', 'CE','C']

In [ ]:
########SCITE as train and test#######################

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.96      0.96      0.96      2663
           E       0.87      0.85      0.86       468
          CE       1.00      0.00      0.00        25
           C       0.85      0.88      0.86       465

    accuracy                           0.93      3621
   macro avg       0.92      0.67      0.67      3621
weighted avg       0.93      0.93      0.93      3621



In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(id2tag)

{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}


In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'CE','O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.932500,0.811865
2,0.620100,0.494064
3,0.224700,0.196857


       preds  gold_data
1          3          1
2          1          1
5          3          1
6          1          1
7          3          1
...      ...        ...
13519      1          1
13520      1          1
13521      3          3
13522      3          3
13523      3          3

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.87      0.88      0.88       468
           C       0.87      0.88      0.88       465
          CE       1.00      0.00      0.00        25
           O       0.96      0.97      0.96      2663

    accuracy                           0.94      3621
   macro avg       0.93      0.68      0.68      3621
weighted avg       0.94      0.94      0.93      3621



Epoch,Training Loss,Validation Loss
1,0.156800,0.192417
2,0.142400,0.166822
3,0.067400,0.179687


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      3          3
13522      3          3
13523      3          3

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.90      0.90      0.90       468
           C       0.86      0.92      0.89       465
          CE       1.00      0.00      0.00        25
           O       0.97      0.97      0.97      2663

    accuracy                           0.95      3621
   macro avg       0.93      0.70      0.69      3621
weighted avg       0.95      0.95      0.94      3621



Epoch,Training Loss,Validation Loss
1,0.050400,0.167772
2,0.036600,0.168044
3,0.020300,0.192468


...

       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      3          3
13522      3          3
13523      3          3

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.92      0.89      0.90       468
           C       0.87      0.93      0.90       465
          CE       0.83      0.40      0.54        25
           O       0.97      0.97      0.97      2663

    accuracy                           0.95      3621
   macro avg       0.90      0.80      0.83      3621
weighted avg       0.95      0.95      0.95      3621



Epoch,Training Loss,Validation Loss
1,0.000100,0.281520
2,0.000100,0.294572
3,0.000100,0.308974


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      3          3
13522      3          3
13523      3          3

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.93      0.87      0.90       468
           C       0.89      0.93      0.91       465
          CE       0.83      0.40      0.54        25
           O       0.97      0.98      0.97      2663

    accuracy                           0.95      3621
   macro avg       0.91      0.79      0.83      3621
weighted avg       0.95      0.95      0.95      3621



Epoch,Training Loss,Validation Loss
1,0.000100,0.326111
2,0.000000,0.348136
3,0.000000,0.372445


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      3          3
13522      3          3
13523      3          3

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.94      0.87      0.90       468
           C       0.89      0.93      0.91       465
          CE       0.83      0.40      0.54        25
           O       0.97      0.98      0.97      2663

    accuracy                           0.95      3621
   macro avg       0.91      0.79      0.83      3621
weighted avg       0.95      0.95      0.95      3621



In [ ]:
#############SCITE as train, Fincausal as test#########################

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# for tokens_test in data_test['words']:
#   s1 = json.dumps(tokens_test)
#   X_raw_test = json.loads(s1)

In [ ]:
# for tokens_test in data_test['ner']:
#   s2 = json.dumps(tokens_test)
#   y_raw_test = json.loads(s2)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]


In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,1.196000,1.011827
2,0.668200,0.562584
3,0.295900,0.233892


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(55650,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(55650,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,0,-100
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
55645,3,-100
55646,0,-100
55647,0,-100
55648,0,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
...,...,...
55505,0,3
55506,0,3
55507,0,3
55508,0,3


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'CE', 3: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.05      1.00      0.10       640
           E       0.68      0.02      0.04      5079
           C       0.92      0.06      0.11      6590

    accuracy                           0.09     12309
   macro avg       0.55      0.36      0.09     12309
weighted avg       0.78      0.09      0.09     12309



In [ ]:
######10 runs

In [ ]:
print(id2tag)

{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}


In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'CE','O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.000000,0.375452
2,0.000000,0.389210
3,0.000000,0.399528


       preds  gold_data
1          3          0
2          3          0
3          3          0
4          3          0
5          3          0
...      ...        ...
55505      3          1
55506      3          1
55507      3          1
55508      3          1
55509      3          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.57      0.06      0.12      5079
           C       0.94      0.10      0.18      6590
          CE       0.00      1.00      0.00         0
           O       0.06      1.00      0.11       640

    accuracy                           0.13     12309
   macro avg       0.39      0.54      0.10     12309
weighted avg       0.74      0.13      0.15     12309



Epoch,Training Loss,Validation Loss
1,0.000000,0.402034
2,0.000200,0.383892
3,0.000100,0.413551


       preds  gold_data
1          3          0
2          3          0
3          3          0
4          3          0
5          3          0
...      ...        ...
55505      3          1
55506      3          1
55507      3          1
55508      3          1
55509      3          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.56      0.05      0.09      5079
           C       0.92      0.13      0.23      6590
          CE       0.00      1.00      0.00         0
           O       0.06      1.00      0.11       640

    accuracy                           0.14     12309
   macro avg       0.38      0.54      0.11     12309
weighted avg       0.73      0.14      0.17     12309



Epoch,Training Loss,Validation Loss
1,0.006700,0.397252
2,0.002100,0.363368
3,0.005600,0.375565


...

       preds  gold_data
1          3          0
2          3          0
3          3          0
4          3          0
5          3          0
...      ...        ...
55505      3          1
55506      3          1
55507      3          1
55508      3          1
55509      3          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.61      0.08      0.14      5079
           C       0.92      0.12      0.22      6590
          CE       0.00      1.00      0.00         0
           O       0.06      0.99      0.11       640

    accuracy                           0.15     12309
   macro avg       0.40      0.55      0.12     12309
weighted avg       0.75      0.15      0.18     12309



Epoch,Training Loss,Validation Loss
1,0.000000,0.384884
2,0.003700,0.450710
3,0.000000,0.386063


       preds  gold_data
1          3          0
2          3          0
3          3          0
4          3          0
5          3          0
...      ...        ...
55505      3          1
55506      3          1
55507      3          1
55508      3          1
55509      3          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.59      0.07      0.12      5079
           C       0.92      0.10      0.19      6590
          CE       0.00      1.00      0.00         0
           O       0.06      1.00      0.11       640

    accuracy                           0.14     12309
   macro avg       0.39      0.54      0.10     12309
weighted avg       0.74      0.14      0.16     12309



Epoch,Training Loss,Validation Loss
1,0.000000,0.397991
2,0.000200,0.424021
3,0.000000,0.390180


       preds  gold_data
1          3          0
2          3          0
3          3          0
4          3          0
5          3          0
...      ...        ...
55505      3          1
55506      3          1
55507      3          1
55508      3          1
55509      3          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.57      0.06      0.11      5079
           C       0.94      0.11      0.20      6590
          CE       0.00      1.00      0.00         0
           O       0.06      1.00      0.11       640

    accuracy                           0.14     12309
   macro avg       0.39      0.54      0.11     12309
weighted avg       0.74      0.14      0.16     12309



In [ ]:
######################SCITE as train and Organizational data as test###########################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_test= tokenize(X_dev_raw)

In [ ]:
for i, x in enumerate(y_dev_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_dev_raw[i][j] = a.replace('CT', 'CE')

In [ ]:
y_dev_raw

[['C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'CE',
  'CE',
  'CE',
  'O',
  'O',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E'],
 ['CE',
  'CE',
  'CE',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'C',
  'C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'CE',
  'CE',
  'CE',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'O',
  'E',
  'E',
  'E'],
 ['O',
  'O',
  'E',
  'E',
  'E',
  'E',
  'CE',
  'CE',
  'O',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C'],
 ['O',
  'O'

In [ ]:
y_test = encode_tags(tags=y_dev_raw, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,1.196000,1.011827
2,0.668200,0.562584
3,0.295900,0.233892


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(52746,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(52746,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,0,3
2,0,3
3,0,3
4,0,3
5,0,3
...,...,...
52655,0,1
52656,0,1
52657,0,1
52658,0,1


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'CE', 3: 'C'}


In [ ]:
targetname = ['O','E','CE','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.15      0.95      0.26      2107
           E       0.66      0.06      0.10      5662
          CE       1.00      0.00      0.00       871
           C       0.70      0.07      0.12      5694

    accuracy                           0.19     14334
   macro avg       0.63      0.27      0.12     14334
weighted avg       0.62      0.19      0.13     14334



In [ ]:
########10 runs##########################################

In [ ]:
print(id2tag)

{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}


In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'CE','O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.000000,0.430413
2,0.000000,0.450282
3,0.004000,0.418445


       preds  gold_data
1          3          1
2          3          1
3          3          1
4          3          1
5          3          1
...      ...        ...
52655      3          0
52656      3          0
52657      3          0
52658      3          0
52659      3          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.74      0.08      0.14      5662
           C       0.78      0.11      0.19      5694
          CE       0.00      0.00      0.00       871
           O       0.15      0.94      0.26      2107

    accuracy                           0.21     14334
   macro avg       0.42      0.28      0.15     14334
weighted avg       0.63      0.21      0.17     14334



Epoch,Training Loss,Validation Loss
1,0.002600,0.397785
2,0.000000,0.385817
3,0.000000,0.389302


       preds  gold_data
1          3          1
2          3          1
3          3          1
4          3          1
5          3          1
...      ...        ...
52655      3          0
52656      3          0
52657      3          0
52658      3          0
52659      3          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.72      0.10      0.18      5662
           C       0.76      0.12      0.20      5694
          CE       0.00      0.00      0.00       871
           O       0.15      0.92      0.26      2107

    accuracy                           0.22     14334
   macro avg       0.41      0.28      0.16     14334
weighted avg       0.61      0.22      0.19     14334



Epoch,Training Loss,Validation Loss
1,0.000000,0.375432
2,0.000000,0.388054
3,0.000000,0.395272


...

       preds  gold_data
1          3          1
2          3          1
3          3          1
4          3          1
5          3          1
...      ...        ...
52655      3          0
52656      3          0
52657      3          0
52658      3          0
52659      3          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.74      0.09      0.17      5662
           C       0.79      0.10      0.18      5694
          CE       0.00      0.00      0.00       871
           O       0.15      0.92      0.26      2107

    accuracy                           0.21     14334
   macro avg       0.42      0.28      0.15     14334
weighted avg       0.63      0.21      0.17     14334



Epoch,Training Loss,Validation Loss
1,0.000000,0.410791
2,0.000000,0.418553
3,0.000000,0.431188


       preds  gold_data
1          3          1
2          3          1
3          3          1
4          3          1
5          3          1
...      ...        ...
52655      3          0
52656      3          0
52657      3          0
52658      3          0
52659      3          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.73      0.11      0.19      5662
           C       0.77      0.11      0.19      5694
          CE       0.00      0.00      0.00       871
           O       0.15      0.91      0.26      2107

    accuracy                           0.22     14334
   macro avg       0.41      0.28      0.16     14334
weighted avg       0.62      0.22      0.19     14334



Epoch,Training Loss,Validation Loss
1,0.000000,0.441518
2,0.000000,0.462076
3,0.000000,0.468842


       preds  gold_data
1          3          1
2          3          1
3          3          1
4          3          1
5          3          1
...      ...        ...
52655      3          0
52656      3          0
52657      3          0
52658      3          0
52659      3          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'CE', 3: 'O'}
              precision    recall  f1-score   support

           E       0.75      0.09      0.17      5662
           C       0.77      0.11      0.20      5694
          CE       0.00      0.00      0.00       871
           O       0.15      0.92      0.26      2107

    accuracy                           0.22     14334
   macro avg       0.42      0.28      0.16     14334
weighted avg       0.63      0.22      0.18     14334



In [ ]:
################ FinCausal as train and FinCausal as test#########################################

In [ ]:
data_train = pd.read_json('/content/fincausal-extra-all-train.json', lines=True)
data_train.head(4)

,words,ner,id,postfix
0,"[It, found, that, total, U.S., health, care, s...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0001.00026,"[ , , , , , , , , , , , , , , , ..."
1,"[Transat, loss, more, than, doubles, as, it, w...","[E, E, E, E, E, O, C, C, C, C, C, C, C]",0003.00001,"[ , , , , , , , , , , , ]"
2,"[MONTREAL, -, Transat, AT, Inc., 's, third-qua...","[O, O, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00002,"[ , , , , , , , , , , , , , , ,..."
3,"[The, Montreal-based, company, lost, $, 11.0, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00003,"[ , , , , , , , , , , , , , , , ,..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_train = data_train[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_train.A.values], [json.dumps(labels_test) for labels_test in dataset_train.B.values]

X_raw, y_raw = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
id2tag

{0: 'C', 1: 'O', 2: 'E'}

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_dev_raw, X_dev_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_raw_test, X_raw_test]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)


In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())


    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(X_train_raw)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(X_raw_test)

y_train = encode_tags(tags=y_train_raw, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,0.507600,0.460651
2,0.214100,0.212497
3,0.088400,0.141092


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(55650,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(55650,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,1,-100
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
55645,2,-100
55646,2,-100
55647,2,-100
55648,2,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
...,...,...
55505,2,2
55506,2,2
55507,2,2
55508,2,2


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.82      0.86      0.84       640
           E       0.93      0.94      0.93      5079
           C       0.95      0.94      0.95      6590

    accuracy                           0.93     12309
   macro avg       0.90      0.91      0.91     12309
weighted avg       0.93      0.93      0.93     12309



In [ ]:
#########10 runs #######################

In [ ]:
print(id2tag)

{0: 'E', 1: 'C', 2: 'O'}


In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.428400,0.378204
2,0.157000,0.208016
3,0.051400,0.147334


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.95      0.94      0.94      5079
           C       0.96      0.96      0.96      6590
           O       0.86      0.90      0.88       640

    accuracy                           0.95     12309
   macro avg       0.92      0.93      0.93     12309
weighted avg       0.95      0.95      0.95     12309



Epoch,Training Loss,Validation Loss
1,0.055200,0.140579
2,0.019200,0.195683
3,0.008700,0.133607


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.94      0.94      0.94      5079
           C       0.96      0.94      0.95      6590
           O       0.82      0.93      0.87       640

    accuracy                           0.94     12309
   macro avg       0.90      0.94      0.92     12309
weighted avg       0.94      0.94      0.94     12309



Epoch,Training Loss,Validation Loss
1,0.011400,0.198109
2,0.004100,0.140966
3,0.005900,0.133103


...

       preds  gold_data
1          1          0
2          1          0
3          1          0
4          1          0
5          1          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.95      0.92      0.94      5079
           C       0.94      0.96      0.95      6590
           O       0.84      0.89      0.86       640

    accuracy                           0.94     12309
   macro avg       0.91      0.92      0.92     12309
weighted avg       0.94      0.94      0.94     12309



Epoch,Training Loss,Validation Loss
1,0.031400,0.170876
2,0.012000,0.169189
3,0.002900,0.145668


       preds  gold_data
1          1          0
2          1          0
3          1          0
4          1          0
5          1          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.95      0.93      0.94      5079
           C       0.95      0.96      0.95      6590
           O       0.86      0.91      0.88       640

    accuracy                           0.94     12309
   macro avg       0.92      0.93      0.92     12309
weighted avg       0.94      0.94      0.94     12309



Epoch,Training Loss,Validation Loss
1,0.000400,0.191069
2,0.010900,0.147107
3,0.002800,0.143161


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.95      0.94      0.95      5079
           C       0.97      0.96      0.96      6590
           O       0.75      0.91      0.82       640

    accuracy                           0.95     12309
   macro avg       0.89      0.94      0.91     12309
weighted avg       0.95      0.95      0.95     12309



In [ ]:
########Fincausal as train and SCITE as test#################################

In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'O')

In [ ]:
y_raw_test

[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E',
  'O',
  'C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'E', 'O', 'E', 'O', 'O', 'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'E', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'O', 'O', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,0.521200,0.451448
2,0.228600,0.214891
3,0.091000,0.142699


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(13561,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(13561,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,1,-100
1,2,2
2,2,2
3,2,-100
4,2,-100
...,...,...
13556,1,-100
13557,1,-100
13558,1,-100
13559,1,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,2,2
2,2,2
5,2,2
6,2,2
7,2,2
...,...,...
13519,2,2
13520,2,2
13521,2,0
13522,2,0


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.98      0.12      0.22      2688
           E       0.17      0.56      0.27       468
           C       0.24      0.92      0.38       465

    accuracy                           0.28      3621
   macro avg       0.47      0.53      0.29      3621
weighted avg       0.78      0.28      0.24      3621



In [ ]:
##########10 runs ######################################################

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.065400,0.121023
2,0.015500,0.143645
3,0.016400,0.111115


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      2          0
13521      0          1
13522      2          1
13523      2          1

[3621 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           E       0.28      0.87      0.42       465
           C       0.98      0.13      0.24      2688
           O       0.19      0.74      0.31       468

    accuracy                           0.31      3621
   macro avg       0.49      0.58      0.32      3621
weighted avg       0.79      0.31      0.27      3621



Epoch,Training Loss,Validation Loss
1,0.011700,0.205364
2,0.038700,0.128195
3,0.022200,0.112276


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          1
13522      0          1
13523      0          1

[3621 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           E       0.26      0.88      0.40       465
           C       0.98      0.12      0.21      2688
           O       0.19      0.70      0.30       468

    accuracy                           0.29      3621
   macro avg       0.48      0.56      0.30      3621
weighted avg       0.79      0.29      0.25      3621



Epoch,Training Loss,Validation Loss
1,0.005900,0.153997
2,0.004100,0.161881
3,0.003500,0.106967


...

       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      2          0
13520      2          0
13521      2          1
13522      2          1
13523      2          1

[3621 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           E       0.25      0.78      0.38       465
           C       0.98      0.14      0.25      2688
           O       0.19      0.74      0.30       468

    accuracy                           0.30      3621
   macro avg       0.48      0.55      0.31      3621
weighted avg       0.79      0.30      0.27      3621



Epoch,Training Loss,Validation Loss
1,0.016300,0.138050
2,0.003300,0.244212
3,0.001800,0.175323


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      2          0
13520      2          0
13521      2          1
13522      2          1
13523      2          1

[3621 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           E       0.25      0.80      0.38       465
           C       0.99      0.13      0.23      2688
           O       0.19      0.73      0.30       468

    accuracy                           0.29      3621
   macro avg       0.48      0.55      0.30      3621
weighted avg       0.79      0.29      0.26      3621



Epoch,Training Loss,Validation Loss
1,0.003300,0.164951
2,0.041500,0.122144
3,0.008200,0.153968


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          1
13522      0          1
13523      0          1

[3621 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           E       0.23      0.86      0.36       465
           C       0.97      0.12      0.22      2688
           O       0.18      0.58      0.27       468

    accuracy                           0.28      3621
   macro avg       0.46      0.52      0.28      3621
weighted avg       0.77      0.28      0.24      3621



In [ ]:
###################Fincausal as train and Organizational as test##############################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_test= tokenize(X_dev_raw)

In [ ]:
for i, x in enumerate(y_dev_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_dev_raw[i][j] = a.replace('CT', 'O')

In [ ]:
y_test = encode_tags(tags=y_dev_raw, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,0.521200,0.451448
2,0.228600,0.214891
3,0.091000,0.142699


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(52746,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(52746,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,1,-100
1,1,2
2,1,2
3,1,2
4,1,2
...,...,...
52741,1,-100
52742,1,-100
52743,1,-100
52744,1,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,1,2
2,1,2
3,1,2
4,1,2
5,1,2
...,...,...
52655,2,1
52656,2,1
52657,2,1
52658,2,1


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.82      0.26      0.40      2978
           E       0.64      0.74      0.68      5662
           C       0.63      0.76      0.69      5694

    accuracy                           0.65     14334
   macro avg       0.70      0.59      0.59     14334
weighted avg       0.67      0.65      0.63     14334



In [ ]:
##############10 runs###################################################

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(id2tag)

{0: 'C', 1: 'O', 2: 'E'}


In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','O', 'E']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.390500,0.387096
2,0.135600,0.206040
3,0.049000,0.130709


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
52655      0          2
52656      0          2
52657      0          2
52658      0          2
52659      0          2

[14334 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           C       0.70      0.80      0.75      5694
           O       0.84      0.26      0.39      2978
           E       0.69      0.84      0.76      5662

    accuracy                           0.70     14334
   macro avg       0.74      0.63      0.63     14334
weighted avg       0.73      0.70      0.68     14334



Epoch,Training Loss,Validation Loss
1,0.063000,0.119564
2,0.017500,0.150549
3,0.019300,0.121028


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
52655      0          2
52656      0          2
52657      0          2
52658      0          2
52659      0          2

[14334 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           C       0.72      0.81      0.76      5694
           O       0.88      0.25      0.39      2978
           E       0.69      0.86      0.77      5662

    accuracy                           0.72     14334
   macro avg       0.76      0.64      0.64     14334
weighted avg       0.74      0.72      0.69     14334



Epoch,Training Loss,Validation Loss
1,0.010000,0.145130
2,0.007300,0.170218
3,0.007700,0.128957


...

       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
52655      0          2
52656      0          2
52657      0          2
52658      0          2
52659      0          2

[14334 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           C       0.69      0.78      0.73      5694
           O       0.90      0.26      0.40      2978
           E       0.66      0.82      0.73      5662

    accuracy                           0.69     14334
   macro avg       0.75      0.62      0.62     14334
weighted avg       0.72      0.69      0.66     14334



Epoch,Training Loss,Validation Loss
1,0.010300,0.146516
2,0.002000,0.183589
3,0.001700,0.114730


       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
52655      0          2
52656      0          2
52657      0          2
52658      0          2
52659      0          2

[14334 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           C       0.68      0.78      0.73      5694
           O       0.89      0.24      0.38      2978
           E       0.66      0.81      0.73      5662

    accuracy                           0.68     14334
   macro avg       0.74      0.61      0.61     14334
weighted avg       0.71      0.68      0.65     14334



Epoch,Training Loss,Validation Loss
1,0.001300,0.133346
2,0.004600,0.133068
3,0.001600,0.142207


       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
52655      2          2
52656      2          2
52657      2          2
52658      2          2
52659      2          2

[14334 rows x 2 columns]
{0: 'C', 1: 'O', 2: 'E'}
              precision    recall  f1-score   support

           C       0.67      0.80      0.73      5694
           O       0.88      0.25      0.39      2978
           E       0.67      0.79      0.73      5662

    accuracy                           0.68     14334
   macro avg       0.74      0.61      0.62     14334
weighted avg       0.71      0.68      0.66     14334



In [ ]:
#################### Organizational data as train and organizational data as test######################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}


In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_train_final = np.array(X_train_raw)
y_train_final = np.array(y_train_raw)

<ipython-input-15-100920d9c2b5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_train_raw)
<ipython-input-15-100920d9c2b5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_train_raw)


In [ ]:
print(np.shape(X_dev_raw))
print(np.shape(y_dev_raw))
print(np.shape(X_train_raw))

(447,)
(447,)
(1787,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
import os
k=2
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
metrics=[]
i=0
for train_index, val_index in kf.split(X_train_final,y_train_final):
  train_texts, test_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
  train_labels, test_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [test_texts, test_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [train_texts, train_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # print(arr_offset)
        # print(type(arr_offset))

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(train_texts)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(test_texts)

y_train = encode_tags(tags=train_labels, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=test_labels, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=y_train)

<ipython-input-24-2ab7c11a097d>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,1.214300,1.153262
2,0.914300,0.878064
3,0.633900,0.537422


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(118769,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(118769,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,3,-100
1,2,2
2,2,-100
3,2,-100
4,2,-100
...,...,...
118764,2,-100
118765,3,-100
118766,3,-100
118767,3,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,2,2
5,2,2
6,2,2
7,2,2
8,2,2
...,...,...
118667,0,0
118668,0,1
118669,3,3
118670,3,3


In [ ]:
print(id2tag)

{0: 'CT', 1: 'O', 2: 'E', 3: 'C'}


In [ ]:
targetname = ['CT', 'O', 'E', 'C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CT       0.86      0.94      0.90      1780
           O       0.66      0.25      0.36      5216
           E       0.77      0.94      0.84     11316
           C       0.79      0.83      0.81     11362

    accuracy                           0.78     29674
   macro avg       0.77      0.74      0.73     29674
weighted avg       0.76      0.78      0.75     29674



In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(id2tag)

{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','E', 'O','CT']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,1.229000,1.145056
2,0.869300,0.815007
3,0.586200,0.477610


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      1          0
13520      0          0
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.26      0.74      0.39       465
           E       0.20      0.70      0.32       468
           O       0.82      0.19      0.30      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.32      3621
   macro avg       0.32      0.41      0.25      3621
weighted avg       0.66      0.32      0.31      3621



Epoch,Training Loss,Validation Loss
1,0.399700,0.462126
2,0.328200,0.477157
3,0.311800,0.468347


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.26      0.75      0.38       465
           E       0.21      0.65      0.32       468
           O       0.81      0.23      0.36      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.35      3621
   macro avg       0.32      0.41      0.26      3621
weighted avg       0.65      0.35      0.35      3621



Epoch,Training Loss,Validation Loss
1,0.212400,0.502528
2,0.144900,0.569931
3,0.161000,0.651710


...

       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.25      0.79      0.39       465
           E       0.23      0.58      0.33       468
           O       0.82      0.26      0.40      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.37      3621
   macro avg       0.33      0.41      0.28      3621
weighted avg       0.67      0.37      0.38      3621



Epoch,Training Loss,Validation Loss
1,0.006700,1.048944
2,0.002500,1.214205
3,0.003100,1.103367


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          2
13522      1          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.27      0.71      0.39       465
           E       0.23      0.63      0.34       468
           O       0.83      0.31      0.45      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.40      3621
   macro avg       0.33      0.41      0.30      3621
weighted avg       0.67      0.40      0.42      3621



Epoch,Training Loss,Validation Loss
1,0.004000,1.144421
2,0.000800,1.177905
3,0.006200,1.199933


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      1          0
13520      0          0
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.29      0.72      0.41       465
           E       0.22      0.66      0.33       468
           O       0.83      0.27      0.41      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.38      3621
   macro avg       0.33      0.41      0.29      3621
weighted avg       0.68      0.38      0.40      3621



In [ ]:
######################Organizational data as train and SCITE as test##################

In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'CT')

In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
print(id2tag)

{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}


In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','E', 'O', 'CT']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.003200,1.175407
2,0.000600,1.226703
3,0.001800,1.229422


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      1          0
13520      0          0
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.28      0.70      0.40       465
           E       0.22      0.69      0.33       468
           O       0.81      0.25      0.38      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.36      3621
   macro avg       0.33      0.41      0.28      3621
weighted avg       0.66      0.36      0.37      3621



Epoch,Training Loss,Validation Loss
1,0.001100,1.238745
2,0.000200,1.226102
3,0.000700,1.301876


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.28      0.75      0.41       465
           E       0.22      0.63      0.33       468
           O       0.83      0.27      0.41      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.38      3621
   macro avg       0.33      0.41      0.29      3621
weighted avg       0.67      0.38      0.40      3621



Epoch,Training Loss,Validation Loss
1,0.001300,1.297375
2,0.000200,1.328138
3,0.003000,1.331588


...

       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      1          0
13520      0          0
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.29      0.74      0.42       465
           E       0.20      0.72      0.31       468
           O       0.83      0.19      0.31      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.33      3621
   macro avg       0.33      0.41      0.26      3621
weighted avg       0.67      0.33      0.32      3621



Epoch,Training Loss,Validation Loss
1,0.008000,1.344875
2,0.000100,1.362200
3,0.002500,1.405831


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      0          0
13520      0          0
13521      0          2
13522      1          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.30      0.71      0.42       465
           E       0.20      0.77      0.32       468
           O       0.85      0.19      0.31      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.33      3621
   macro avg       0.34      0.42      0.26      3621
weighted avg       0.69      0.33      0.32      3621



Epoch,Training Loss,Validation Loss
1,0.001800,1.336949
2,0.000000,1.347486
3,0.001600,1.451475


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      1          0
13520      0          0
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.29      0.70      0.41       465
           E       0.20      0.77      0.32       468
           O       0.84      0.18      0.30      2663
          CT       0.00      0.00      0.00        25

    accuracy                           0.32      3621
   macro avg       0.33      0.41      0.26      3621
weighted avg       0.68      0.32      0.32      3621



In [ ]:
############Organizational data as train and FinCausal as test ###########################

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]


In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','CT', 'O','E']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.001800,1.385012
2,0.000000,1.380919
3,0.000000,1.397301


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
55505      2          0
55506      0          0
55507      2          0
55508      0          0
55509      2          0

[12309 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.93      0.62      0.74      6590
          CT       0.65      0.67      0.66      5079
           O       0.09      0.30      0.13       640
           E       0.00      1.00      0.00         0

    accuracy                           0.62     12309
   macro avg       0.42      0.65      0.38     12309
weighted avg       0.77      0.62      0.68     12309



Epoch,Training Loss,Validation Loss
1,0.002100,1.393723
2,0.000000,1.419042
3,0.000100,1.474067


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
55505      2          0
55506      0          0
55507      2          0
55508      0          0
55509      2          0

[12309 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.89      0.72      0.80      6590
          CT       0.72      0.69      0.70      5079
           O       0.11      0.29      0.16       640
           E       0.00      1.00      0.00         0

    accuracy                           0.68     12309
   macro avg       0.43      0.67      0.42     12309
weighted avg       0.78      0.68      0.73     12309



Epoch,Training Loss,Validation Loss
1,0.001800,1.433438
2,0.000200,1.447090
3,0.000100,1.475177


...

       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
55505      2          0
55506      0          0
55507      1          0
55508      1          0
55509      2          0

[12309 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.91      0.62      0.74      6590
          CT       0.65      0.65      0.65      5079
           O       0.09      0.31      0.14       640
           E       0.00      1.00      0.00         0

    accuracy                           0.62     12309
   macro avg       0.41      0.65      0.38     12309
weighted avg       0.76      0.62      0.67     12309



Epoch,Training Loss,Validation Loss
1,0.000100,1.453470
2,0.002000,1.464917
3,0.000400,1.466180


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
55505      2          0
55506      0          0
55507      2          0
55508      0          0
55509      2          0

[12309 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.91      0.66      0.76      6590
          CT       0.68      0.64      0.66      5079
           O       0.10      0.34      0.15       640
           E       0.00      1.00      0.00         0

    accuracy                           0.63     12309
   macro avg       0.42      0.66      0.39     12309
weighted avg       0.77      0.63      0.69     12309



Epoch,Training Loss,Validation Loss
1,0.000400,1.445551
2,0.000000,1.434412
3,0.000000,1.424613


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
55505      2          0
55506      0          0
55507      1          0
55508      0          0
55509      2          0

[12309 rows x 2 columns]
{0: 'C', 1: 'E', 2: 'O', 3: 'CT'}
              precision    recall  f1-score   support

           C       0.89      0.66      0.76      6590
          CT       0.67      0.63      0.65      5079
           O       0.10      0.34      0.15       640
           E       0.00      1.00      0.00         0

    accuracy                           0.63     12309
   macro avg       0.42      0.66      0.39     12309
weighted avg       0.76      0.63      0.68     12309



In [ ]:
!python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
           --train_file /content/content/data/fin_causal_2022/fincausal-extra-all-train.json  \
           --validation_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
           --test_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
           --output_dir /content/content/data/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16  #--classifier_dropout 0.1